<a href="https://colab.research.google.com/github/YoheiFukuhara/recommender-system/blob/main/03_preprocess2dict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle

import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/Recommend/output/'
df = pd.read_csv(BASE_PATH+'very_small_rating.csv')

In [ ]:
cutoff = int(0.8*len(df))
df_train, df_test = train_test_split(df, train_size=cutoff)

In [ ]:
user2movie = {}
movie2user = {}
usermovie2rating = {}
count = 0

In [ ]:
%%time
def update_user2movie_and_movie2user(row):
    global count
    count += 1
    if count % 500000 == 0:
        print('processed: %.3f' % (float(count)/cutoff))
    i = int(row.userId)
    j = int(row.movie_idx)
    if i not in user2movie:
        user2movie[i] = [j]
    else:
        user2movie[i].append(j) #リストで複数持つ

    if j not in movie2user:
        movie2user[j] = [i]
    else:
        movie2user[j].append(i) #リストで複数持つ
    
    usermovie2rating[(i,j)] = row.rating

df_train.apply(update_user2movie_and_movie2user, axis=1)

processed: 0.116
processed: 0.232
processed: 0.348
processed: 0.464
processed: 0.580
processed: 0.695
processed: 0.811
processed: 0.927
CPU times: user 2min 22s, sys: 1.69 s, total: 2min 23s
Wall time: 2min 24s


In [ ]:
%%time
usermovie2rating_test = {}
count = 0
def update_usermovie2rating_test(row):
    global count
    count += 1
    if count % 100000 == 0:
        print('processed: %.3f' % (float(count)/cutoff))
    i = int(row.userId)
    j = int(row.movie_idx)    
    usermovie2rating_test[(i,j)] = row.rating

df_test.apply(update_usermovie2rating_test, axis=1)

processed: 0.023
processed: 0.046
processed: 0.070
processed: 0.093
processed: 0.116
processed: 0.139
processed: 0.162
processed: 0.185
processed: 0.209
processed: 0.232
CPU times: user 37.5 s, sys: 876 ms, total: 38.4 s
Wall time: 38.4 s


In [ ]:
def dump_as_pickle(variable, file):
    with open(BASE_PATH+file+'.pickle', 'wb') as f:
        pickle.dump(variable, f)

dump_as_pickle(user2movie, 'user2movie')
dump_as_pickle(movie2user, 'movie2user')
dump_as_pickle(usermovie2rating, 'usermovie2rating')
dump_as_pickle(usermovie2rating_test, 'usermovie2rating_test')